In [1]:
import json
from os.path import join as pjoin
import cv2

In [2]:
data_root = 'E:\\Mulong\\Datasets\\gui\\rico\\combined\\all'
result_root = 'E:\\Temp\\data\\rico-layout\\result'
ground_truth = js.load(open())

In [5]:
f = open('select.txt', 'r')
for img_name in f.readlines()[:200]:
    img_name = img_name.replace('\n', '')
    print(img_name)
    
    visualize_results(img_name)
    key = visualize_ground_truth(img_name, js)
    if key is not None and key != -1 and chr(key) == 'q':
        break
cv2.destroyAllWindows()

2114


In [6]:
name = '2114'
visualize_results(img)
visualize_ground_truth(img, js)
cv2.destroyAllWindows()

In [32]:
for img_name in img_names:
    img_data = js[img_name]
    print(img_data.keys())

dict_keys(['horizontal_groups', 'ui', 'multitab_groups', 'pageindcator_groups', 'bounds', 'vertical_groups', 'list_groups', 'labels', 'clickables'])
dict_keys(['horizontal_groups', 'ui', 'multitab_groups', 'pageindcator_groups', 'bounds', 'vertical_groups', 'list_groups', 'labels', 'clickables'])
dict_keys(['horizontal_groups', 'ui', 'multitab_groups', 'pageindcator_groups', 'bounds', 'vertical_groups', 'list_groups', 'labels', 'clickables'])
dict_keys(['horizontal_groups', 'ui', 'multitab_groups', 'pageindcator_groups', 'bounds', 'vertical_groups', 'list_groups', 'labels', 'clickables'])
dict_keys(['horizontal_groups', 'ui', 'multitab_groups', 'pageindcator_groups', 'bounds', 'vertical_groups', 'list_groups', 'labels', 'clickables'])
